In [1]:
from math import *


class Node:
    def __init__(self, name):
        self.name = str(name)
        self.next = None

    def __str__(self):
        nextnode = 'NULL' if not self.next else self.next.name
        return str(self.name) + '->' + nextnode


def get_distance_matrix(data):
    matrix = []
    for i in range(len(data)):
        tmp = []
        for j in range(len(data)):
            tmp.append(get_dist(data[i], data[j]))
        matrix.append(tmp)
    
    return matrix


def get_dist(p1, p2):
    dist = 0
    for i in range(len(p1)):
        dist = dist + (p1[i]-p2[i])**2

    return sqrt(dist)


def agglomerative(data, min_dist):
    clusters = [Node(_) for _ in range(len(data))]
    matrix = get_distance_matrix(data)

    for _ in range(len(data) - 1):
        matrix, i, j, dist = get_closest_point(matrix)
        if dist <= min_dist:
            clusters[i].next = clusters[j]
        else:
            break

    classes = []
    for i in range(len(data)):
        clas = clusters[i]
        while clas.next != None:
            clas = clas.next
        classes.append(clas.name)

    uniq = list(set(classes))
    mapped = {}
    for i in range(len(uniq)):
        mapped[uniq[i]] = i

    for i in range(len(classes)):
        classes[i] = mapped[classes[i]]
    return classes
    

def get_closest_point(matrix):
    INF = 99999999
    min_dist = INF
    ii = 0
    jj = 1
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i != j and matrix[i][j] < min_dist:
                min_dist = matrix[i][j]
                ii = i
                jj = j
                matrix[i][j] = INF
                matrix[j][i] = INF

    return matrix, ii, jj, min_dist

In [2]:
class K_Means:
    def __init__(self, clusters):
        self.clusters = clusters
        self.max_iter = 100
    
    def fit(self, data):        
        # Init centroids
        self.centroids = data.copy()
        np.random.shuffle(self.centroids)
        self.centroids = self.centroids[:self.clusters]
        
        for i in range(self.max_iter):
            distance = np.sqrt(((data - self.centroids[:, np.newaxis])**2).sum(axis=2))
            classes = np.argmin(distance, axis=0)
            self.centroids = np.array([data[classes==k].mean(axis=0) for k in range(self.centroids.shape[0])])
            
    def predict(self, data):
        distance = np.sqrt(((data - self.centroids[:, np.newaxis])**2).sum(axis=2))
        classes = np.argmin(distance, axis=0)
        return classes

In [3]:

class DBSCAN:
    def __init__(self, min_pts, epsilon):
        self.min_pts = min_pts
        self.epsilon = epsilon
        self.NOT_VISITED_LABEL = -99
        self.NOISE_LABEL = -1

    def label_dbscan(self, data):
        labels = [self.NOT_VISITED_LABEL]*len(data) 
        cluster = 0

        for datum_idx in range(0, len(data)):
            if not (labels[datum_idx] == self.NOT_VISITED_LABEL):
                continue

            neighbors = self.neighbor_points(data, datum_idx)

            if len(neighbors) < self.min_pts:
                labels[datum_idx] = self.NOISE_LABEL
            else: 
                self.flood_fill(data, labels, datum_idx, neighbors, cluster)
                cluster += 1

        return labels


    def flood_fill(self, data, labels, datum_idx, neighbors, cluster):
        labels[datum_idx] = cluster
        i = 0
        while i < len(neighbors):           
            neighbor = neighbors[i]
            
            if labels[neighbor] == self.NOISE_LABEL:
                labels[neighbor] = cluster
            elif labels[neighbor] == self.NOT_VISITED_LABEL:
                labels[neighbor] = cluster
                new_neighbors = self.neighbor_points(data, neighbor)
                if len(new_neighbors) >= self.min_pts:
                    neighbors = neighbors + new_neighbors
                    
            i = i + 1


    def neighbor_points(self, data, datum_idx):
        neighbors = []

        for curr_idx in range(0, len(data)):
            if self.find_euclidean_distance(data[datum_idx], data[curr_idx]) < self.epsilon:
                neighbors.append(curr_idx)
            # print(find_euclidean_distance(data[datum_idx], data[curr_idx]))

        return neighbors

    def find_euclidean_distance(self, a, b):
        dist_square = 0
        for i in range(len(a)):
            dist_square = dist_square + ((a[i] - b[i])**2)

        return dist_square**(.5)
    
    def predict(self, X_train, labels, X_test):
        pred_labels = []
        for point_test in X_test:
            dist = 999999
            idx = -1
            i = 0
            for point in X_train:
                temp = self.find_euclidean_distance(point_test, point)
                if temp < dist:
                    dist = temp
                    idx = i 
                i = i + 1
                
            pred_labels.append(labels[idx])
        
        return pred_labels
                

In [4]:
def pre_process(path_file):
    df = pd.read_csv(path_file, sep=',', names=['A1', 'A2', 'A3', 'A4', 'kelas'])
    
    obj_df = df.select_dtypes(include=['object']).copy()
    obj_df['kelas'] = obj_df['kelas'].astype('category')
    obj_df['kelas'] = obj_df['kelas'].cat.codes
    
    df['kelas'] = obj_df['kelas']
    
    return df

In [5]:
import pandas as pd
import numpy as np

df = pre_process('iris.data')
df

,A1,A2,A3,A4,kelas
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [6]:
# Split into train and test data

X = df.drop(['kelas'], axis = 1)
y = df['kelas']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train.shape

(120, 4)

In [7]:
kmeans_scratch = K_Means(3)
kmeans_scratch.fit(X_train)

In [8]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

y_pred = kmeans_scratch.predict(X_test)

pred = [0]*6

pred[0] = y_pred
pred[1] = [0 if x==0 else 2 if x==1 else 1 for x in pred[0]]
pred[2] = [1 if x==0 else 0 if x==1 else 2 for x in pred[0]]
pred[3] = [1 if x==0 else 2 if x==1 else 0 for x in pred[0]]
pred[4] = [2 if x==0 else 0 if x==1 else 1 for x in pred[0]]
pred[5] = [2 if x==0 else 1 if x==1 else 0 for x in pred[0]]

max_pred = [accuracy_score(y_test, pred[0]), accuracy_score(y_test, pred[1]), accuracy_score(y_test, pred[2]),
           accuracy_score(y_test, pred[3]), accuracy_score(y_test, pred[4]), accuracy_score(y_test, pred[5])]

# Find index with maximum accuracy
best_idx = max_pred.index(max(max_pred))

print(f"accuracy_score scratch:  {accuracy_score(y_test, pred[best_idx])}")
print(f"f1_score scratch: {f1_score(y_test, pred[best_idx], average=None)}")
print(f"recall_score scratch: {recall_score(y_test, pred[best_idx], average=None)}")

accuracy_score scratch:  0.9
f1_score scratch: [1.         0.89655172 0.66666667]
recall_score scratch: [1.  1.  0.5]


In [9]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 3)
kmeans.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [10]:
kmeans_pred = kmeans.predict(X_test)

print(kmeans_pred)
print(y_test)

[2 2 1 0 1 0 1 2 2 2 0 2 2 2 2 1 2 2 1 1 2 2 1 1 2 1 1 2 2 1]
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]


In [11]:
pred = [0]*6

pred[0] = kmeans_pred
pred[1] = [0 if x==0 else 2 if x==1 else 1 for x in pred[0]]
pred[2] = [1 if x==0 else 0 if x==1 else 2 for x in pred[0]]
pred[3] = [1 if x==0 else 2 if x==1 else 0 for x in pred[0]]
pred[4] = [2 if x==0 else 0 if x==1 else 1 for x in pred[0]]
pred[5] = [2 if x==0 else 1 if x==1 else 0 for x in pred[0]]

max_pred = [accuracy_score(y_test, pred[0]), accuracy_score(y_test, pred[1]), accuracy_score(y_test, pred[2]),
           accuracy_score(y_test, pred[3]), accuracy_score(y_test, pred[4]), accuracy_score(y_test, pred[5])]

# Find index with maximum accuracy
best_idx = max_pred.index(max(max_pred))

print(f"accuracy_score scratch:  {accuracy_score(y_test, pred[best_idx])}")
print(f"f1_score scratch: {f1_score(y_test, pred[best_idx], average=None)}")
print(f"recall_score scratch: {recall_score(y_test, pred[best_idx], average=None)}")

accuracy_score scratch:  0.9
f1_score scratch: [1.         0.89655172 0.66666667]
recall_score scratch: [1.  1.  0.5]


In [12]:
dbscan = DBSCAN(min_pts = 10, epsilon = .9)
labels = dbscan.label_dbscan(X_train)
print(labels)

pred = [0]*6

pred[0] = dbscan.predict(X_train, labels, X_test)
pred[1] = [0 if x==0 else 2 if x==1 else 1 for x in pred[0]]
pred[2] = [1 if x==0 else 0 if x==1 else 2 for x in pred[0]]
pred[3] = [1 if x==0 else 2 if x==1 else 0 for x in pred[0]]
pred[4] = [2 if x==0 else 0 if x==1 else 1 for x in pred[0]]
pred[5] = [2 if x==0 else 1 if x==1 else 0 for x in pred[0]]

max_pred = [accuracy_score(y_test, pred[0]), accuracy_score(y_test, pred[1]), accuracy_score(y_test, pred[2]),
           accuracy_score(y_test, pred[3]), accuracy_score(y_test, pred[4]), accuracy_score(y_test, pred[5])]

# Find index with maximum accuracy
best_idx = max_pred.index(max(max_pred))

print(f"accuracy_score scratch:  {accuracy_score(y_test, pred[best_idx])}")
print(f"f1_score scratch: {f1_score(y_test, pred[best_idx], average=None)}")
print(f"recall_score scratch: {recall_score(y_test, pred[best_idx], average=None)}")

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, -1, 0, -1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1]
accuracy_score scratch:  0.8
f1_score scratch: [1.     0.8125 0.    ]
recall_score scratch: [1. 1. 0.]


/home/ebarhami/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
from sklearn.cluster import DBSCAN as DBSCAN2

clustering = DBSCAN2(eps=.9, min_samples=10).fit(X_train)
print(clustering.labels_)

dbscan2 = DBSCAN(min_pts = 10, epsilon = .9)

pred2 = [0]*6
pred2[0] = dbscan2.predict(X_train, clustering.labels_, X_test)
pred2[1] = [0 if x==0 else 2 if x==1 else 1 if x==2 else -1 for x in pred2[0]]
pred2[2] = [1 if x==0 else 0 if x==1 else 2 if x==2 else -1 for x in pred2[0]]
pred2[3] = [1 if x==0 else 2 if x==1 else 0 if x==2 else -1 for x in pred2[0]]
pred2[4] = [2 if x==0 else 0 if x==1 else 1 if x==2 else -1 for x in pred2[0]]
pred2[5] = [2 if x==0 else 1 if x==1 else 0 if x==2 else -1 for x in pred2[0]]

max_pred2 = [accuracy_score(y_test, pred2[0]), accuracy_score(y_test, pred2[1]), accuracy_score(y_test, pred2[2]),
           accuracy_score(y_test, pred2[3]), accuracy_score(y_test, pred2[4]), accuracy_score(y_test, pred2[5])]

# Find index with maximum accuracy
best_idx2 = max_pred.index(max(max_pred2))

print(f"accuracy_score scratch:  {accuracy_score(y_test, pred[best_idx2])}")
print(f"f1_score scratch: {f1_score(y_test, pred[best_idx2], average=None)}")
print(f"recall_score scratch: {recall_score(y_test, pred[best_idx2], average=None)}")

[ 0  0  1  0  0  0  1  0  0  0  0  1  0  1  1  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  1  0  0  1  1  0  1  0  0
  1  0  0  0  1  0  0  0  0  1  1  0  0  1  0  1  0  0  1  1  0  1  1  1
  0  0  0  1  1  1  0  0  1  1  0  1 -1  0 -1  0  1  0  1  0  1  1  0  1
  0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  1  1  1  0  0  0  1]
accuracy_score scratch:  0.8
f1_score scratch: [1.     0.8125 0.    ]
recall_score scratch: [1. 1. 0.]


/home/ebarhami/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
